프로덕션 환경으로 코드를 가져다 놓는 것은 애플리케이션을 개발에서 고객에게 가져가는 마지막 단계다. 프로덕션 환경으로 애플리케이션을 배포하ㅡㄴ 작업은 상당히 중요하지만 소프트웨어 아키텍트의 체크리스트에서 빠뜨릴 떄가 있다.

시스템이 개발 환경에서 동작하는 것처럼 프로덕션 환경에서도 잘 동작하리라 가정하지만 이는 매우 치명적인 실수다. 대부분의 프로덕션 시스템은 개발 환경의 구성과 매우 다르다. 이용할 수 있는 많은 최적화 기법과 디버깅, 개발 환경에서 당연하다고 여겨지는 것들이 프로덕션 환경에서는 사용할 수 없는 경우가 있다.

애플리케이션을 프로덕션 환경에 배포하는 일은 정밀한 과학이라기보다 예술에 가깝다. 시스템의 배포 복잡도는 시스템을 개발한 언어, 시스템의 런타임 이식성과 성능, 구성 파라미터 수, 시스템의 동종 환경이나 이종 환경으로의 배포 여부, 배포의 지리적 분포, 배포 자동화 도구, 다른 기타 요소에 영향을 받는다.

최근 몇 년 동안, 파이썬은 오픈소스 언어로써 자동화 수준이 높아졌으며 프로덕션 시스템으로의 패키지 배포를 지원하고 있다. 파이썬은 내장된 풍부한 가용성 및 서드 파티 지원 도구와 더불어, 프로덕션 배포와 배포 시스템을 최신으로 유지하는 고통과 번거로움을 감소시켰다.

배포 가능한 시스템 및 배포 용이성의 개념을 간단히 살펴보고, 파이썬을 사용해 프로덕션 시스템에서 실행되는 애플리케이션의 배포와 유지보수를 쉽게 하기 위해, 아키텍트가 레퍼토리에 추가할 수 있는 파이썬 애플리케이션의 배포 도구, 프로세스를 알아본다. 빈번하게 시스템을 중지하지 않아도 아키텍트가 프로덕션 시스템을 정상적이고 안전하게 유지하기 위해 적용할 수 있는 기법과 모범 사례도 살펴볼 것이다.

## 배포 용이성

소프트웨어 시스템의 배포 용이성은 소프트웨어를 개발 환경에서 프로덕션 환경으로 쉽게 가져갈 수 있는 정도다. 배포 용이성은 코드를 개발 환경에서 프로덕션 환경으로 배포하는 데 필요한 분리 단계 관점 즉, 공수 대비 인력-시간이나 복잡도 측면에서 측정할 수 있다.

개발 시스템이나 스테이징 시스템에서 잘 동작하는 코드가 프로덕션 시스템에서 유사한 방법으로 잘 동작할 것이라는 가정은 자주하는 실수다. 이런 실수가 일어나는 이유는 가끔 개발 시스템이 프로덕션 시스템과 매우 다른 요구사항을 갖고 있기 때문이다.

### 배포 용이성에 영향을 주는 요소들

* 최적화 및 디버깅: 개발 시스템에서는 코드의 최적화를 하지 않는 것이 일반적이다. 파이썬 같이 코드가 런타임에서 인터프리터 퍼리돼 실행될 때, 보통 프로그래머는 예외가 발생할 때 많은 양의 역추적 정보를 생성할 수 있도록 디버그 설정을 활성화한다. 대개 파이썬 인터프러티의 최적화는 비활성화돼 있다. 반면 프로덕션 시스템에서는 최적화가 활성화되고 디버깅은 비활성화된다. 이와 유사한 방식으로 코드를 동작시키려면 추가적인 구성이 필요하다. 물론 특정 상황에서는 프로그램을 최적화되지 않은 상태로 실행시키는 것보다 최적화 정도에 따라 다르게 동작시키는 것도 가능하다.

* 의존성과 버전: 일반적으로 개발 환경에는 개발자가 작업할 수 있는 다양한 애플리케이션을 실행하기 위한 풍부한 개발 라이브러리와 지원 라이브러리가 설치돼 있다. 대부분 개발자는 최신 코드로 작업하기 때문에 라이브러리 의존성을 갖고 있을 수 있다. 프로덕션 시스템은 미리 컴파일된 의존성 목록과 버전으로 주의 깊게 준비돼야 한다. 보통 프로덕션 시스템으로의 배포는 성숙하거나 안정된 버전만 지정한다. 따라서 개발자가 다운스트림에서 사용했던 기능을 불안정한 버전이나 버그 수정에 의존했다면, 프로덕션 환경에서 기능이 의도대로 동작하지 않는다는 사실을 너무 늦게 발견하기도 한다. 또 다른 문제를 문서화되지 않는 의존성이나 소스코드에서 컴파일해야 하는 의존성들인데 처음 배포할 때 문제를 일으키기도 한다.

* 리소스 구성과 액세스 권한: 개발 시스템과 프로덕션 시스템은 리소스 접근 수준과 권한, 세부 사항이 다르다. 개발 시스템은 로컬 데이터베이스를 가질 수 있지만 프로덕션 시스템은 애플리케이션과 데이터베이스 시스템을 별도로 호스팅하는 경향이 있다. 개발 시스템은 표준 구성 파일을 사용할 수 있다. 반면, 프로덕션 환경의 구성은 특정 스크립트를 사용해 호스트나 환경에 특화되게 생성할 수 있다. 비슷한 형태로 프로덕션 환경의 애플리케이션은 특정 사용자/그룹에서는 더 낮은 권한으로 실행될 수도 있지만 개발 환경에서는 프로그램을 root나 superuser로 실행한다. 사용자 권한과 구성의 불일치는 리소스 액세스에 영향을 줄 수 있으며 소프트웨어가 개발 환경에서 정상적으로 동작할 떄도 프로덕션 환경에서는 실패의 원인이 될 수 있다.

* 이기종 프로덕션 환경: 일반적으로 코드는 동질성을 갖는 개발 환경에서 개발된다. 그러나 코드는 프로덕션 환경의 이기종 시스템에 배포할 수 있어야 한다. 예를 들어 소프트웨어는 리눅스에서 개발될 수 있지만, 고객은 윈도우 관련 배포 요구사항을 갖고 있을 수 있다. 배포의 복잡성은 환경의 이질성 비례해 증가한다. 코드가 프로덕션 환경으로 가기 전, 잘 관리된 스테이징 환경과 테스팅 환경이 필요하다. 이기종 시스템은 각 대상 시스템의 아키텍처를 위한 별도의 의존성 목록을 관리해야 하기 때문에 의존성 관리를 더 어렵게 만든다.

* 보안: 개발 환경과 테스트 환경은 시간을 절약하고 테스팅을 위한 구성의 복잡도를 줄이기 위해 보안 측면에서 차이를 두는 것이 일반적이다. 예컨대 웹 애플리케이션에서 빠른 프로그래밍과 편리한 테스팅을 위해 특별한 개발 환경 플래그로 로그인이 필요한 경로를 비활성화할 수 있다. 유사하게 개발 환경에서 사용되는 데이터베이스 시스템, 웹 애플리케이션 로그인 시스템들은 일상적인 기억과 사용 편의를 위해 추측하기 쉬운 패스워드를 사용한다. 역할 기반으로 권한을 부여하는 것은 테스팅을 쉽게 만들기 위해 무시될 수 있다. 프로덕션 환경에서 보안은 중요하기 때문에 개발 환경과느 반대여야 한다.보안 측면에서 로그인이 필요한 경로는 개발 환경과 반대로 강력한 패스워드를 사용해야 한다. 그리고 역할 기반으로 권한을 부여해야 한다. 이것은 개발환경에서는 동작하지만 프로덕션 환경에서 실패하는 미묘한 버그의 원인이 된다.

## 소프트웨어 배포 아키텍처 계층

코드를 개발할 때 테스팅 및 프로덕션 호나경까지의 복잡성 방지를 위해, 프로덕션 환경으로 배포하기 전의 애플리케이션의 수명 주기 각 단계는 다중 계층 아키텍처를 사용한다.

* 개발/테스트/스테이지/프로덕션: 전통적인 4계층 아키텍처
    * 개발자는 자신의 코드를 단위 테스트와 개발자 테스트가 실행되는 개발 환경에 넣는다. 개발 환경에서는 항상 최신 트렁크나 첨단 코드를 사용한다. 대부분 이 환경을 건너뛰고 개발자 컴퓨터의 로컬 설정으로 대체한다.
    * 그 후 QA나 테스팅 엔지니어가 테스트 환경에서 블랙박스 기법으로 소프트웨어를 테스트하는데 테스트 환경에서 성능 테스트를 할 수도 있다. 이 환경은 코드 업데이트 측면에서 언제나 개발 환경보다 늦다. 일반적으로 내부 출시, 태그, 코드 덤프가 개발 환경과 QA 환경을 동기화하는 데 사용된다.
    * 스테이징 환경은 프로덕션 환경과 가능한 유사하게 구성된다. 스테이징 환경은 나중에 프로덕션 환경에서 발생할 수 있는 문제를 발견하기 위해 가능한 배포 호나경과 유사한 환경에서 소프트웨어를 테스트하는 사전 프로덕션 환경이다. 스트레스 테스트나 부하 테스트는 스테이징 환경에서 실행된다. 데브옵스 엔지니어는 스테이징 환경에서 배포 자동화 스크립트, cron 작업, 시스템 구성을 검증하기 위한 테스트를 할 수 있다.
    * 프로덕션 환경은 스테이징 환경에서 푸시된 배포 소프트웨어가 테스트되는 마지막 계층이다. 배포 단계에서 대부분 스테이징 환경과 프로덕션 환경을 동일하게 사용하고 단순히 한 계츠에서 다른 계층으로 전환한다.
    
* 개발 및 테스트 환경/스테이지 환경/프로덕션 환경: 이것은 개발 환경에다 테스트 환경까지 이중의 의무를 수행하게 하는 이전 계층 구조의 변형이다. 시스템은 코드가 최소한 일주일에 한 번 프로덕션 환경으로 푸시되는 애자일 소프트웨어의 개발 실천방법을 적용하고 있고, 별도의 테스팅 환경을 관리하고 유지할 공간이나 시간이 없는 회사에서 사용한다. 개발자가 프로그래밍을 위해 자신의 컴퓨터를 사용하는 것 처럼 별도의 개발 환경이 없을 때는 테스팅 환경도 로컬 환경이 된다.

* 개발 및 테스트 환경/스테이지 및 프로덕션 환경: 이 설정은 사용되는 여러 서버와 떠불어 스테이징 환경과 프로덕션 환경이 정확하게 같아. 스테이징 환경에서 시스템이 테스트되고 검증되면 호스트를 전환해 간단하게 프로덕션 환경으로 푸시된다. 현재 프로덕션 시스템은 스테이징 시스템으로 전환되고, 스테이징 시스템은 프로덕션 시스템으로 전환된다.

## 파이썬에서 소프트웨어 배포

### 파이썬 코드 패키징

파이썬에서 소스코드를 패키징 하는 주된 방법은 setup.py 파일을 작성하는 것이다. 그러면 소스코드를 내장된 distutils 라이브러리나 더 정교하고 다양한 기능이 많은 setuptools 프레임워크를 통해 패키징할 수 있다.

### Pip

pip는 파이썬에서 패키지 설치를 위한 쵸준이자 권장 도구다.

### Virtualenv

Virtualenv는 개발자가 로컬로 개발하기 위한 샌드박스 파이썬 환경을 생성하는 도구다. 나란히 개발하는 서로 다른 두 개의 애플리케이션을 위해 특정 라이브러리나 프레임워크의 두 가지 버전을 유지해야 한다고 해 보자.

시스템에 파이썬에 관련된 모든 것을 설치한다면 주어진 시점에는 한 버전만 유지할 수 있다. 

Virtualenv는 권한과 버전 문제를 한 번에 해결한다. 

### Virtualenv와 pip

애플리케이션을 위한 가상 환경을 설정하고 필요한 패키지를 설치하면 애플리케이션의 의존성과 의존성 버전의 정보를 생성하는 것이 좋다. pip를 사용해 다음 명령어를 통해 실행하면 된다.

pip freeze | tee requirements.txt

배포 미러링을 위해 서버에 설정 사항이 복제된다. 

### PyPI

pip는 어디에서 패키지를 가져올까?

PyPI(Python Package Index)는 웹상에서 서드 파티 파이썬 패키지에 대한 메타 데이터를 호스팅하는 공식 저장소다. 이름에서 보듯 PyPI는 서버에 게시되고 색인처리되는 메타데이터를 갖는 웹상의 파이썬 패키지 색인이다. pypi.python.org에 호스팅된다.

패키지들은 파이썬의 패키징 및 배포 도구, distutils와 setuptools를 통해 PyPI로 제출되며 패키지 메타데이터를 게시하기 위한 후크를 갖고 있다. s


### 패브릭을 사용한 원격 배포

패브릭은 파이썬으로 작성된 명령행 도구 및 라이브러리다. 패브릭은 SSH 프로토콜상의 잘 정의된 래퍼 세트를 통해 서버로의 원격 배포 자동화를 돕는다. 패브릭은 ssh-wrapper 라이브러리, paramiko를 사용한다.

패브릭은 파이썬 2 버전에서만 동작한다. 그러나 파이썬 3 버전 모두에서 동작하는 포크 버전인 Fabric3도 있다.

패브릭을 사용할 때 데브옵스 사용자는 fabfile.py라고 하는 fabfile 안의 파이썬 함수로 원격 시스템 관리자 명령을 배포한다.

패브릭은 원격 시스템이 배포를 수행하는 사용자 머신의 ssh 공개키를 갖고 구성됐을 때 가장 잘 동작하며 이때는 사용자 이름과 패스워드를 제공할 필요가 없다.

다음은 서버에 대한 원격 배포 예제다.

In [3]:
from fabric.api import run

def remote_install(application):
    print("Installing", application)
    run("sudo pip install " + application)

In [4]:
# pip install fabric3

In [5]:
# fab remote_install:mandelbrot -H mylinode

데브옵스 엔지니어와 시스템 관리자는 여러 서버의 다양한 시스템으로의 애플리키에션 배포 작업을 자동화하기 위해 미리 정의된 fabfile 세트를 사용할 수 있다.

파이썬으로 작성됐지만 패브릭은 모든 유형의 원격 서버 관리와 구성 작업의 자동화에 사용할 수 있다.



### 앤서블을 이용환 원격 배포

애서블은 파이썬으로 작성된 구성 관리 및 배포 도구다. 앤서블은 플레이북으로 불리는 관리 단위를 쉽게 조합할 수 있는 태스크를 통해 오케스트레이션을 지원하는 스크립트를 갖고 있는 SSH의 래퍼로 생각할 수 있다. 플레이북은 호스트 그룹을 일련의 역할 세트와 매핑한다.

앤서블은 시스템과 환경 벙조가 태스크 실행 전에 수집되는 팩트를 이용한다. 앤서블은 원하는 결과를 얻기 위해 태스크 실행 전에 상태를 바꿔야 하는지 여부를 확인하기 위해 팩트를 이용한다.

앤서블 태스크를 서버에 반복되는 방식으로 실행하는 것이 안전하다. 잘 작성된 앤서블 태스크는 원격 시스템에서 몇 가지 부작용이 있다는 점에서 멱등성을 갖는다.

앤서블은 파이썬으로 작성됐으며 pip를 사용해 설치할 수 있다.

앤서블은 태스크를 수행하는 호스트 정보를 유지하기 위해 /etc/ansible/hosts라는 자체 호스트 파일을 사용한다.

일반적인 앤서블 호스트 파일은 다음과 비슷하다.

다음은 pip를 통해 webkaffe라고 하는 원격 호스트에 일부 파이썬 패키지를 설치하는 dependencies.yaml로 명명된 ansible-playbook의 코드다.

앤서블은 원격 의존성을 쉽게 효과적으로 관리하는 방법이며, 멱등성을 갖는 플레이북으로 인해 태스크에서 패브릭보다 훨씬 더 유용하다.

### 슈퍼바이저를 이용한 원격 데몬 관리하기

슈퍼바이저는 유닉스와 유닉스 계열 시스템에서 프로세스를 관리하는 데 유용한 클라이언트/서버 시스템이다. 슈퍼바이저는 주로 supervisord 서버 데몬 프로세스와 서버와 상호 작용하는 supervisorctl 명령행 클라이언트로 구성된다.

슈퍼바이저는 9001 포트를 통해 접근 가능한 기본 웹 서버와 함께 제공된다. 웹 서버에서는 실행되는 프로세스의 상태를 볼 수 있으며, 이 인터페이스를 통해 프로세스 시작과 종료를 할 수 있다. 슈퍼바이저는 윈도우 버전에서는 실행되지 않는다.

슈퍼바이저는 파이썬을 사용해 작성된 애플리케이션이다. 따라서 pip를 통해 설치가 가능하다. 슈퍼바이저는 파이썬 2버전에서만 실행된다.

## 배포 패턴과 모범 사례

* 지속적인 배포: 소프트웨어가 언제든지 실제 환경으로 배포할 준비가 돼 있는 배치 모델이다. 지속적인 전달은 개발, 테스팅, 스테이징을 포함한 계층이 지속적으로 통합될 때만 가능하다. 지속적인 배포 모델에서는 하루에 여러 개의 프로덕션 배포가 배포 파이프라인을 통해 자동으로 수행될 수 있다. 계속 늘어나는 변경 사항을 배포하기 떄문에 지속적인 배포 모드는 배포 위험을 최소화한다. 애자일 개발 방식에서 지속적인 배포는 개발 및 테스트가 끝나자마자 프로덕션 환경에서 실제 코드를 볼 수 있기 때문에 고객이 진행 상황을 직접 추적할 수 있다. 사용자 피드백을 받아들여 코드와 기능을 더 빠르게 반복할 수 있게 하는 장점도 추가된다.

* 블루 그린 배포: 블루-그린 배포는 거의 똑같은 구 개의 프로덕션 환경을 유지한다. 두 개의 환경이 주어진 상황에서 한 환경은 라이브 환경이다. 새로운 배포 변경 사항을 다른 환경에서 준비하고 테스트가 끝나 실제 환경으로 갈 준비가 되면 시스템을 전환한다. 그린은 액티브 상태가 되고 블루는 백업된다. 새로운 배포에서 문제가 있을 때는 라우터나 로드 밸러서를 또 다른 환경으로 전환하면 되기 떄문에 블루-그린 배포는 배포 위험을 상당히 감소히킨다. 일반적으로 전형적인 블루-그린 배포 시스템에서, 한 시스템은 프로덕션 환경이고 다른 환경은 스테이징 환경으로 둘 사이의 역할은 서로 전환된다.

* 카나리아 출시: 고객 전체를 대상으로 변경사항을 배포하기 전에 일부 사용자만 제한해 소프트웨어의 변경사항을 테스트할 때 사용하는 방법이다. 간단한 방법은 도그푸딩으로 직원에게 변경 사항을 내부적으로 먼저 배포하는 것이다. 베타 테스팅, 인구 지리적 위치 프로파일 기반으로 사용자를 선택하는 방법이다. 잘못 관리되는 기능에 대한 갑작스러운 고객의 반응으로부터 회사를 보호하며, 점진적인 방식으로 부하를 관리하고 용량을 확장할 수 있다. 

* 버킷 테스팅(A/B 테스팅): 서로 다른 두 가지 버전의 애플리케이션이나 웹 페이지를 프로덕션 환경으로 배포하고, 어떤 버전이 더 인기가 있고 사용자가 어느 버전에 더 많이 참여하고 있는지 테스트하는 기법이다. 

* 유도된 혼란: 실패나 가용성 수준의 복원력을 테스트하기 위해 의도적으로 오류를 주입하거나 프로덕션 배포 시스테므이 일부를 비활성화하는 방법이다.